In [ ]:
from io import BytesIO
import tempfile
import zipfile
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
from pathlib import Path
import concurrent.futures
from glob import glob
import geopandas as gpd

# Funçoes

In [ ]:
def download_files_from_ftp(url: str) -> None:
    """
    Baixa um arquivo do FTP e retorna seu conteúdo em memória.

    Parâmetros:
    url (str): URL do arquivo a ser baixado.

    Retorna:
    Arquivo será baixado e extraido na pasta "data/input"
    """
    base_path = Path("data")
    path_input = base_path / "input"

    response = requests.get(url, stream=True)
    response.raise_for_status()

    save_path = path_input / url.split('/')[-1]

    path_input.mkdir(parents=True, exist_ok=True)

    with open(save_path, "wb") as fd:
        for chunk in response.iter_content(chunk_size=128):
            fd.write(chunk)

    with zipfile.ZipFile(save_path) as z:
        z.extractall(path_input)

def get_sp_distrito_from_csv(path_csv: str) -> pd.DataFrame:

  name_file = path_csv.split('/')[-1]

  temp_df = pd.read_csv(path_csv, encoding='iso-8859-1', sep=";", dtype=str)

  df_sp_distritos = temp_df[temp_df.CD_DIST.str.contains("3550308")]

  return df_sp_distritos

def excel_sheet_to_df( path_dicionario: str, sheet_name: str) -> pd.DataFrame:

  temp_df_sheet = pd.read_excel(path_dicionario, sheet_name=sheet_name)

  temp_df_sheet = temp_df_sheet[['Variável', 'Descrição']]

  return temp_df_sheet

# Download

In [ ]:
url_agregados_por_distrito_csv = 'http://ftp.ibge.gov.br/Censos/Censo_Demografico_2022/Agregados_por_Setores_Censitarios/Agregados_por_Distrito_csv/'
url_sp_distrito_geo = 'http://ftp.ibge.gov.br/Censos/Censo_Demografico_2022/Agregados_por_Setores_Censitarios/malha_com_atributos/distritos/shp/UF/SP/SP_distritos_CD2022.zip'
url_dicionario_de_dados = 'http://ftp.ibge.gov.br/Censos/Censo_Demografico_2022/Agregados_por_Setores_Censitarios/dicionario_de_dados_agregados_por_setores_censitarios.xlsx'

response = requests.get(url_agregados_por_distrito_csv)

soup = BeautifulSoup(response.text)

links_agregados_por_distrito_csv = [url_agregados_por_distrito_csv + a.get('href') for a in soup.select('td a') if a.get('href').count('zip')] # Coleta o link dos Arquivos

links_agregados_por_distrito_csv.extend([url_sp_distrito_geo, url_dicionario_de_dados]) # Adicionando tabela Distrito Geografico e Dicionario a lista de download

with concurrent.futures.ThreadPoolExecutor() as executor:
  executor.map(download_files_from_ftp, links_agregados_por_distrito_csv)



# Separar distritos de SP

In [ ]:
# Define tabela com as GeoGrafias
geo_df_sp = gpd.read_file('/content/data/input/SP_distritos_CD2022.shp', columns=['CD_DIST', 'NM_DIST', 'geometry'])
geo_df_sp = geo_df_sp[geo_df_sp.CD_DIST.str.contains("3550308")]

# Carrega o Dicionario e Prepara ele para o Rename
path_dicionario = '/content/data/input/dicionario_de_dados_agregados_por_setores_censitarios.xlsx'
sheet_names = ['Dicionário Básico', 'Dicionário não PCT', 'Dicionário PCT - Indígenas', 'Dicionário PCT - Quilombolas']

df_dicionario = pd.concat([excel_sheet_to_df(path_dicionario, sheet_name) for sheet_name in sheet_names])

dicionario_to_rename = dict(zip(df_dicionario.Variável, df_dicionario.Descrição))

# Carrega e trata tabela final

regex_glob = '/content/data/input/*.csv'

csv_files = glob(regex_glob)

temp_dfs = [get_sp_distrito_from_csv(csv_path) for csv_path in csv_files]

temp_dfs.append(geo_df_sp) # Adicionando tabela Distrito Geografico

first_temp_df = temp_dfs.pop(0) # Seleciona primeira tabela e remove ela da lista

for temp_df in temp_dfs:

  temp_df.drop('NM_DIST', axis=1, inplace=True)

  first_temp_df = pd.merge(
      first_temp_df,
      temp_df,
      how="left",
      on="CD_DIST"
  )

first_temp_df.columns = first_temp_df.columns.str.upper() # Coloca todas as colunas como Upper para manipulação de nomes

# Tratar ordem e nome das colunas
columns_to_sorted = first_temp_df.columns[first_temp_df.columns.str.startswith('V')].sort_values().tolist()
first_columns = first_temp_df.columns[~first_temp_df.columns.str.startswith('V')].tolist()
column_order = first_columns + columns_to_sorted

first_temp_df = first_temp_df[column_order]

first_temp_df = first_temp_df.rename(columns=dicionario_to_rename)

first_temp_df = first_temp_df.loc[:,~first_temp_df.columns.duplicated()] # Remove Colunas duplicadas.

first_temp_df.GEOMETRY = first_temp_df.GEOMETRY.astype(str) # Transforma Geometry em string para salvar em Parquet

first_temp_df.to_excel('ibge_censo_setor_censitario.xlsx', index=False) # Salvar para excel
first_temp_df.to_parquet('ibge_censo_setor_censitario.parquet', index=False) # Salvar para Parquet